In [1]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/022/581/195/GCF_022581195.2_ilHelZeax1.1/GCF_022581195.2_ilHelZeax1.1_genomic.gff.gz
!gunzip ./GCF_022581195.2_ilHelZeax1.1_genomic.gff.gz
!mv GCF_022581195.2_ilHelZeax1.1_genomic.gff genomic.gff

--2025-06-18 18:09:54--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/022/581/195/GCF_022581195.2_ilHelZeax1.1/GCF_022581195.2_ilHelZeax1.1_genomic.gff.gz
Распознаётся ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)… 130.14.250.13, 130.14.250.7, 130.14.250.12, ...
Подключение к ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 6435348 (6,1M) [application/x-gzip]
Сохранение в: «GCF_022581195.2_ilHelZeax1.1_genomic.gff.gz»

GCF_022581195.2_ilH 100%[===================>]   6,14M   486KB/s    за 9,3s    

2025-06-18 18:10:04 (679 KB/s) - «GCF_022581195.2_ilHelZeax1.1_genomic.gff.gz» сохранён [6435348/6435348]



In [2]:
!head genomic.gff

##gff-version 3
#!gff-spec-version 1.21
#!processor NCBI annotwriter
#!genome-build ilHelZeax1.1
#!genome-build-accession NCBI_Assembly:GCF_022581195.2
#!annotation-source NCBI Helicoverpa zea Annotation Release 100
##sequence-region NC_061452.1 1 15512169
##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=7113
NC_061452.1	RefSeq	region	1	15512169	.	+	.	ID=NC_061452.1:1..15512169;Dbxref=taxon:7113;Name=1;chromosome=1;collected-by=USDA-ARS Genetics and Sustainability Agricultural Research Unit%2C Starkville%2C MS%2C USA;collection-date=2011;country=USA: Starkville%2C MS;dev-stage=pupa;gbkey=Src;genome=chromosome;isolate=HzStark_Cry1AcR;isolation-source=laboratory colony at USDA ARS Southern Insect Management Unit%2C Stoneville%2C MS;mol_type=genomic DNA;sex=male;tissue-type=Whole body
NC_061452.1	Gnomon	gene	2693	4723	.	+	.	ID=gene-LOC124645826;Dbxref=GeneID:124645826;Name=LOC124645826;gbkey=Gene;gene=LOC124645826;gene_biotype=protein_coding


In [3]:
!rm 'gene.bed' 'exon.bed' 'CDS.bed' 'mRNA.bed'

with open('genomic.gff', 'r') as f:
    for ind, line in enumerate(f.readlines()):
        if ind < 7:
            continue
        if line.startswith('#'):
            continue
        feature = line.split()[2]
        if not feature in ['mRNA', 'CDS', 'gene', 'exon']:
            continue
        file_name = f'{feature}.bed'
        with open(file_name, 'a') as g:
            add = line.split()[0] + '\t' + line.split()[3] + '\t' + line.split()[4] + '\t' + 'name\tscore\t' + line.split()[6] + '\n'
            g.write(add)

rm: gene.bed: No such file or directory
rm: exon.bed: No such file or directory
rm: CDS.bed: No such file or directory
rm: mRNA.bed: No such file or directory


In [6]:
!bedtools subtract -a gene.bed -b exon.bed > intron.bed

In [7]:
from Bio import SeqIO

!rm 'lengthes.txt'

input_file = "genomic.fna"

for record in SeqIO.parse(input_file, "fasta"):
    name, sequence = record.id, str(record.seq)
    with open('lengthes.txt', 'a') as f:
        f.write(f'{name}\t{len(sequence)}\n')

!head lengthes.txt

rm: lengthes.txt: No such file or directory
NC_061452.1	15512169
NC_061453.1	15061584
NC_061454.1	14695159
NC_061455.1	14165716
NC_061456.1	14051263
NC_061457.1	14023251
NC_061458.1	13649923
NC_061459.1	13649271
NC_061460.1	13615264
NC_061461.1	13258385


In [8]:
!bedtools flank -l 1000 -r 0 -i gene.bed -g lengthes.txt -s > upstream.bed

In [9]:
!bedtools flank -r 200 -l 0 -i gene.bed -g lengthes.txt -s > downstream.bed

In [10]:
!bedtools complement -i gene.bed -g lengthes.txt > intergenic.bed

In [ ]:
import pandas as pd

results = ['G-quadruplex.bed', 'results_zhunt.bed', 'zdnabert_predictions.bed']
places = [ 'exon', 'intron', 'upstream', 'downstream', 'intergenic']
counts = pd.DataFrame(columns=['G4', 'zhunt', 'zdnabert', 'G4_prob', 'zhunt_prob', 'zdnabert_prob'],
                      index = ['exon', 'intron', 'upstream', 'downstream', 'intergenic'])

for r in results:
    name = f'new_{r}'
    !cp {r} {name}

for p in places:
    name_place = f'{p}.bed'
    for ind, algo in enumerate(['G4', 'zhunt', 'zdnabert']):
        name_res = f'new_{results[ind]}'
        name_out = f'cnt_{p}_{algo}.bed'
        with open(name_res, 'r') as f:
            before = len(f.readlines())
        !bedtools intersect -a {name_res} -b {name_place} -wa > {name_out}
        !bedtools subtract -a {name_res} -b {name_out} -A > tmp.bed
        !rm {name_res}
        !mv tmp.bed {name_res}
        with open(name_res, 'r') as f:
            after = len(f.readlines())
        counts.loc[p, algo] = before - after

for ind, algo in enumerate(['G4', 'zhunt', 'zdnabert']):
    name_res = f'new_{results[ind]}'
    !rm {name_res}
    for p in places:
        name_out = f'cnt_{p}_{algo}.bed'
        !rm {name_out}

for p in places:
    for t in ['G4', 'zhunt', 'zdnabert']:
        counts.loc[p, t + '_prob'] = counts.loc[p, t] / counts[t].sum()

counts

,G4,zhunt,zdnabert,G4_prob,zhunt_prob,zdnabert_prob
exon,1722,269443,27074,0.106758,0.166794,0.283041
intron,7923,802370,37805,0.491197,0.496693,0.395227
upstream,449,53385,2858,0.027836,0.033047,0.029879
downstream,62,9514,364,0.003844,0.005889,0.003805
intergenic,5974,480711,27553,0.370366,0.297576,0.288049


In [6]:
import pandas as pd

results = ['G-quadruplex.bed', 'results_zhunt.bed', 'zdnabert_predictions.bed']
places = [ 'exon', 'intron', 'upstream', 'downstream', 'intergenic']
segments = pd.DataFrame(columns=['G4', 'zhunt', 'zdnabert', 'G4_prob', 'zhunt_prob', 'zdnabert_prob'],
                      index = ['exon', 'intron', 'upstream', 'downstream', 'intergenic'])

for p in places:
    name_place = f'{p}.bed'
    for ind, algo in enumerate(['G4', 'zhunt', 'zdnabert']):
        name_res = f'{results[ind]}'
        !bedtools intersect -wa -a {name_place} -b {name_res} | uniq > tmp
        in_count = len(open('tmp').readlines())
        segments.loc[p, algo] = in_count
        segments.loc[p, algo + '_prob'] = in_count / len(open(name_place).readlines())

segments

,G4,zhunt,zdnabert,G4_prob,zhunt_prob,zdnabert_prob
exon,2949,29436,38960,0.012168,0.12146,0.160759
intron,5598,28187,12458,0.053897,0.271381,0.119944
upstream,613,5235,2895,0.036606,0.312612,0.172877
downstream,100,1273,443,0.005972,0.076018,0.026454
intergenic,2997,7488,5395,0.220904,0.551927,0.397656


In [ ]:
counts.to_csv('counts.csv')
segments.to_csv('segments.csv')